# Feature Selection

In [107]:
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt



def selectkbest(indep_X,dep_Y,n):
        test = SelectKBest(score_func=chi2, k=n)
        fit1= test.fit(indep_X,dep_Y)
        # summarize scores       
        selectk_features = fit1.transform(indep_X)
        return selectk_features
    
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    
def r2_prediction(regressor,X_test,y_test):
     y_pred = regressor.predict(X_test)
     from sklearn.metrics import r2_score
     r2=r2_score(y_test,y_pred)
     return r2
 
def Linear(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2   
    
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'linear')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
    
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'rbf')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     

def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training setC
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     

def random(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2 
    
    
def selectk_regression(acclin,accsvml,accsvmnl,accdes,accrf): 
    
    dataframe=pd.DataFrame(index=['ChiSquare'],columns=['Linear','SVMl','SVMnl','Decision','Random'])

    for number,idex in enumerate(dataframe.index):
        
        dataframe['Linear'][idex]=acclin[number]       
        dataframe['SVMl'][idex]=accsvml[number]
        dataframe['SVMnl'][idex]=accsvmnl[number]
        dataframe['Decision'][idex]=accdes[number]
        dataframe['Random'][idex]=accrf[number]
    return dataframe
    

In [108]:
dataset=pd.read_csv("cars.csv",index_col=None)



In [109]:
df2=dataset

df2 = pd.get_dummies(df2, drop_first=True)

indep_X=df2.drop('Price',axis=1)
dep_Y=df2['Price']

In [130]:
kbest=selectkbest(indep_X,dep_Y,5)      

acclin=[]
accsvml=[]
accsvmnl=[]
accdes=[]
accrf=[]



In [131]:
X_train, X_test, y_train, y_test=split_scalar(kbest,dep_Y)  
for i in kbest:   
    r2_lin=Linear(X_train,y_train,X_test)
    acclin.append(r2_lin)
    
    r2_sl=svm_linear(X_train,y_train,X_test)    
    accsvml.append(r2_sl)
    
    r2_NL=svm_NL(X_train,y_train,X_test)
    accsvmnl.append(r2_NL)
    
    r2_d=Decision(X_train,y_train,X_test)
    accdes.append(r2_d)
    
    r2_r=random(X_train,y_train,X_test)
    accrf.append(r2_r)
    
    
result=selectk_regression(acclin,accsvml,accsvmnl,accdes,accrf)


In [129]:
#result4

,Linear,SVMl,SVMnl,Decision,Random
ChiSquare,0.318288,-0.057863,-0.058022,0.756889,0.830664


In [116]:
#result6

,Linear,SVMl,SVMnl,Decision,Random
ChiSquare,0.28075,-0.057863,-0.058021,0.830794,0.832506


In [119]:
#result7

,Linear,SVMl,SVMnl,Decision,Random
ChiSquare,0.28075,-0.057863,-0.058021,0.77123,0.827131


In [132]:
#result5

,Linear,SVMl,SVMnl,Decision,Random
ChiSquare,0.302174,-0.057863,-0.058022,0.753303,0.85583


In [126]:
#result3

,Linear,SVMl,SVMnl,Decision,Random
ChiSquare,0.402017,-0.057863,-0.058016,0.764175,0.856087


In [133]:
#from sklearn.model_selection import train_test_split
#x_train,x_test,y_train,y_test=train_test_split(indep_X,dep_Y,test_size=0.30,random_state=0)

# Model Creation

In [134]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
param_grid= {'n_estimators':[1, 10,50], 'criterion': ['squared_error','absolute_error','friedman_mse','poisson']}
grid=GridSearchCV(RandomForestRegressor(),param_grid,refit=True,verbose=3,n_jobs=-1)
grid.fit(indep_X,dep_Y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error',
                                       'friedman_mse', 'poisson'],
                         'n_estimators': [1, 10, 50]},
             verbose=3)

In [135]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.020886,0.006957,0.011323,0.006105,squared_error,1,"{'criterion': 'squared_error', 'n_estimators': 1}",0.663788,0.898478,0.852202,0.983944,0.977995,0.875281,0.116754,7
1,0.089315,0.005710,0.012872,0.003677,squared_error,10,"{'criterion': 'squared_error', 'n_estimators':...",0.487299,0.969022,0.988499,0.981406,0.986513,0.882548,0.197740,6
2,0.350204,0.013268,0.022754,0.003808,squared_error,50,"{'criterion': 'squared_error', 'n_estimators':...",0.474149,0.977216,0.991672,0.983263,0.989818,0.883224,0.204601,3
3,0.027037,0.003004,0.006450,0.001770,absolute_error,1,"{'criterion': 'absolute_error', 'n_estimators'...",0.287599,0.864217,0.710644,0.828593,0.988032,0.735817,0.240931,12
4,0.135697,0.008699,0.009037,0.001556,absolute_error,10,"{'criterion': 'absolute_error', 'n_estimators'...",0.604937,0.984721,0.982645,0.982597,0.982007,0.907382,0.151225,1
5,0.606113,0.023397,0.025954,0.004499,absolute_error,50,"{'criterion': 'absolute_error', 'n_estimators'...",0.589259,0.977525,0.990839,0.975606,0.985272,0.903700,0.157316,2
6,0.016554,0.004461,0.008415,0.004544,friedman_mse,1,"{'criterion': 'friedman_mse', 'n_estimators': 1}",0.523867,0.921914,0.959053,0.949016,0.917127,0.854195,0.165921,9
7,0.073751,0.002058,0.008242,0.002962,friedman_mse,10,"{'criterion': 'friedman_mse', 'n_estimators': 10}",0.488860,0.984213,0.975472,0.976107,0.990934,0.883117,0.197211,4
8,0.326103,0.026070,0.020255,0.004824,friedman_mse,50,"{'criterion': 'friedman_mse', 'n_estimators': 50}",0.478441,0.978397,0.987224,0.982624,0.988519,0.883041,0.202332,5
9,0.018227,0.005766,0.009986,0.003025,poisson,1,"{'criterion': 'poisson', 'n_estimators': 1}",0.262213,0.959223,0.768229,0.866023,0.955347,0.762207,0.259587,11
